In [1]:
input_cardinality=20
output_cardinality=20

In [2]:
import torch
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import sklearn.model_selection as sk
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [3]:
#import pytorch_lightning as pl
import time
import copy
from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow


2023-06-14 14:23:44.212476: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 14:23:44.442493: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-14 14:23:44.838638: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/toor/Desktop/catkin_ism/devel/lib:/opt/ros/noetic/lib
2023-06-14 14:23:44.838694: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dle

In [4]:
torch.manual_seed(0)
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)
tensorflow.random.set_seed(1234)

In [5]:
#from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
#from pytorch_lightning.loggers import TensorBoardLogger

In [6]:
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")

In [7]:
class DATA:
    def __init__(self, data_path):
        self.data_path = data_path
        self.filenames = os.listdir(DATA_PATH)
        self.filenames = [filename for filename in self.filenames if filename.endswith(".csv")]
        self.len_filenames = len(self.filenames)

    def get_data(self):
        self.data = []
        for i in range(len(self.filenames)):
            self.data.append(self.read_data(i))
        return self.data

    def read_data(self,i):
        return pd.read_csv(self.data_path+"/"+self.filenames[i])


    def process_data(self, scaler, input_cardinality=20, output_cardinality=10, cols=['ThetaXHG']):
        self.datasamples = self.get_data()
        X = []
        y = []
        X_scaled = []
        y_scaled = []
        for i, datasample in enumerate(self.datasamples):
            #for j in range(datasample[cols].shape[0]-output_cardinality):
            #    print('Input' +str(j)+ ' to '+str(j+input_cardinality) + 'Output: '+str(j+input_cardinality)+ ' to '+str(j+input_cardinality+output_cardinality))
            #Append to X (TO MAKE A LIST OF LISTS), [[0.019,1.02,...., upto input_cardinality]]
            #Append to Y (TO MAKE A LIST OF LISTS), [[0.019,1.02,...., upto output_cardinality]]
            #start of Y will be ahead of end of X by 1

            for j in range(datasample.shape[0]-output_cardinality-input_cardinality+1):
                X.append(datasample[cols].iloc[j:j+input_cardinality].to_numpy())
                y.append(datasample[cols].iloc[j+input_cardinality: j+input_cardinality+output_cardinality].to_numpy())
                X_scaled.append(scaler.transform(datasample[cols].iloc[j:j+input_cardinality].to_numpy()))
                y_scaled.append(scaler.transform(datasample[cols].iloc[j+input_cardinality: j+input_cardinality+output_cardinality].to_numpy()))

        #return self.datasamples, X, y
        #print(np.array(X).shape,np.array(y).shape,np.array(X_scaled).shape, np.array(y_scaled).shape)
        return self.datasamples, np.array(X), np.array(y), np.array(X_scaled), np.array(y_scaled)


    def train_test_split(self, X, y, split={'train':0.8,'val':0.2, 'test':0.5}):
        #print(X)
        self.X_train, self.X_val_test, self.y_train, self.y_val_test = sk.train_test_split(X, y, test_size=split['val'] , random_state=43)
        self.X_test, self.X_val, self.y_test, self.y_val = sk.train_test_split(self.X_val_test, self.y_val_test, test_size=split['test'] , random_state=43)

        return (self.X_train, self.y_train), (self.X_val, self.X_test), (self.X_test, self.y_test)

    def keras_train_test_split(self, X, y, split={'train':0.9,'test':0.1}):
        #print(X)
        self.X_train_val, self.X_test, self.y_train_val, self.y_test = sk.train_test_split(X, y, test_size=split['test'] , random_state=43)

        return (self.X_train_val, self.y_train_val), (self.X_test, self.y_test)



    def getscaler(self,cols):
        self.datasamples = self.get_data()
        for i, sample in enumerate(self.datasamples):
            if i ==0:
                features = pd.DataFrame(sample[cols])
            else:
                features = pd.DataFrame.append(features, sample[cols]) #sample[cols]

        #scaling
        scaler = MinMaxScaler(feature_range=(-1,1))
        scaler = scaler.fit(features)
        features_scaled = pd.DataFrame(scaler.transform(features), index = features.index, columns = cols)
        return scaler

        """
        #convert back to datasamples
        #save to X
        #save to X_scaled


        #Split treain test and validation

        for i in range(features.shape[0]):
            self.datasamples = []




        return features, features_scaled
        """



In [8]:
class THETADATASET(Dataset):
    #convert to pytorch dataset
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        sequence, output = self.X[idx], self.y[idx]
        return (torch.from_numpy(sequence.reshape(-1)), torch.from_numpy(output.reshape(-1)))
        #return dict(sequence=torch.tensor(sequence.reshape(-1),dtype=torch.float64), label=torch.tensor(output.reshape(-1),dtype=torch.float64))

In [9]:
class THETADATASETLOADER():
    def __init__(self, data, batchsize, bs_val):
        super().__init__()
        self.data = data
        self.batchsize = batchsize
        self.train_dataset = THETADATASET(self.data.X_train, self.data.y_train)
        self.val_dataset = THETADATASET(self.data.X_val, self.data.y_val)
        self.test_dataset = THETADATASET(self.data.X_test, self.data.y_test)
        self.train_dataloader = DataLoader(self.train_dataset, batch_size = self.batchsize, shuffle= False, num_workers=0)
        self.val_dataloader = DataLoader(self.val_dataset, batch_size = bs_val, shuffle= False, num_workers=0)
        self.test_dataloader = DataLoader(self.test_dataset, batch_size = 1, shuffle= False, num_workers=0)

In [10]:
epochs=200
batch_size = 16
bs_val = 16
DATA_PATH = "../DATASET/TRANSORMER_DATA"
data = DATA(DATA_PATH)
cols=['ThetaXHG']
scaler = data.getscaler(cols)
input_shape = (input_cardinality,1)

In [11]:
samples, X, y, X_scaled, y_scaled = data.process_data(scaler, input_cardinality=input_cardinality, output_cardinality=output_cardinality, cols=['ThetaXHG'])
(X_train_val, y_train_val), (X_test, y_test) = data.keras_train_test_split(X_scaled,y_scaled,  split={'train':0.9,'test':0.1})

In [12]:
X_train_val.shape, y_train_val.shape,  X_test.shape, y_test.shape

((425, 20, 1), (425, 20, 1), (48, 20, 1), (48, 20, 1))

In [13]:
class TRANSFORMER:
    def __init__(self):
        pass

    def definition(self,input_shape, output_cardinality, head_size,num_heads,ff_dim,num_transformer_blocks,mlp_units,dropout=0,mlp_dropout=0):
        inputs = keras.Input(shape=input_shape)
        x = inputs
        for _ in range(num_transformer_blocks):
            x = self.transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

        x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
        for dim in mlp_units:
            x = layers.Dense(dim, activation="relu")(x)
            x = layers.Dropout(mlp_dropout)(x)
        outputs = layers.Dense(output_cardinality)(x)
        return keras.Model(inputs, outputs)

    def transformer_encoder(self,inputs, head_size, num_heads, ff_dim, dropout=0):
        # Normalization and Attention
        x = layers.LayerNormalization(epsilon=1e-6)(inputs)
        x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
        x = layers.Dropout(dropout)(x)
        res = x + inputs

        # Feed Forward Part
        x = layers.LayerNormalization(epsilon=1e-6)(res)
        x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
        x = layers.Dropout(dropout)(x)
        x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
        return x + res

In [14]:
"""
class TRANSFORMER_ENCODER_MODEL:
    def __init__(self):
        pass

    def definition(self,input_shape,head_size,num_heads,ff_dim,num_transformer_blocks,mlp_units,dropout=0,mlp_dropout=0):
        inputs = keras.Input(shape=input_shape)
        x = inputs
        for _ in range(num_transformer_blocks):
            x = self.transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

        x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
        for dim in mlp_units:
            x = layers.Dense(dim, activation="relu")(x)
            x = layers.Dropout(mlp_dropout)(x)
        outputs = layers.Dense(1)(x)
        return keras.Model(inputs, outputs)

    def transformer_encoder(self,inputs, head_size, num_heads, ff_dim, dropout=0):
        # Normalization and Attention
        x = layers.LayerNormalization(epsilon=1e-6)(inputs)
        x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
        x = layers.Dropout(dropout)(x)
        res = x + inputs

        # Feed Forward Part
        x = layers.LayerNormalization(epsilon=1e-6)(res)
        x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
        x = layers.Dropout(dropout)(x)
        x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
        return x + res
"""

'\nclass TRANSFORMER_ENCODER_MODEL:\n    def __init__(self):\n        pass\n\n    def definition(self,input_shape,head_size,num_heads,ff_dim,num_transformer_blocks,mlp_units,dropout=0,mlp_dropout=0):\n        inputs = keras.Input(shape=input_shape)\n        x = inputs\n        for _ in range(num_transformer_blocks):\n            x = self.transformer_encoder(x, head_size, num_heads, ff_dim, dropout)\n\n        x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)\n        for dim in mlp_units:\n            x = layers.Dense(dim, activation="relu")(x)\n            x = layers.Dropout(mlp_dropout)(x)\n        outputs = layers.Dense(1)(x)\n        return keras.Model(inputs, outputs)\n\n    def transformer_encoder(self,inputs, head_size, num_heads, ff_dim, dropout=0):\n        # Normalization and Attention\n        x = layers.LayerNormalization(epsilon=1e-6)(inputs)\n        x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)\n        x = 

In [15]:
"""
net = LSTMNET(batch_size=batch_size,input_len=input_cardinality,output_len=output_cardinality)
net = net.to(device)
net = net.double()
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(),lr=1e-1)
"""

'\nnet = LSTMNET(batch_size=batch_size,input_len=input_cardinality,output_len=output_cardinality)\nnet = net.to(device)\nnet = net.double()\ncriterion = nn.MSELoss()\noptimizer = optim.Adam(net.parameters(),lr=1e-1)\n'

In [16]:
transformer_model = TRANSFORMER()
transformer_predictor = transformer_model.definition(input_shape, output_cardinality,head_size=256, num_heads=4, ff_dim=4, num_transformer_blocks=4, mlp_units=[128], mlp_dropout=0.4,dropout=0.25)

2023-06-14 14:24:06.860607: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-14 14:24:06.862839: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/toor/Desktop/catkin_ism/devel/lib:/opt/ros/noetic/lib
2023-06-14 14:24:06.863216: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/toor/Desktop/catkin_ism/devel/lib:/opt/ros/noetic/lib
2023-06-14 14:24:06.863225: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned abov

In [17]:
transformer_predictor.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20, 1)]      0           []                               
                                                                                                  
 layer_normalization (LayerNorm  (None, 20, 1)       2           ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 multi_head_attention (MultiHea  (None, 20, 1)       7169        ['layer_normalization[0][0]',    
 dAttention)                                                      'layer_normalization[0][0]']    
                                                                                              

 mbda)                                                            'tf.__operators__.add_5[0][0]'] 
                                                                                                  
 layer_normalization_7 (LayerNo  (None, 20, 1)       2           ['tf.__operators__.add_6[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv1d_6 (Conv1D)              (None, 20, 4)        8           ['layer_normalization_7[0][0]']  
                                                                                                  
 dropout_7 (Dropout)            (None, 20, 4)        0           ['conv1d_6[0][0]']               
                                                                                                  
 conv1d_7 (Conv1D)              (None, 20, 1)        5           ['dropout_7[0][0]']              
          

In [18]:
transformer_predictor.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=1e-4))
#model.summary()

In [20]:
stats = transformer_predictor.fit(X_train_val,y_train_val,epochs=epochs,batch_size=batch_size, validation_split = 0.1)


Epoch 1/200
24/24 [==============================] - 15s 206ms/step - loss: 0.3786 - val_loss: 0.3465
Epoch 2/200
24/24 [==============================] - 4s 150ms/step - loss: 0.3092 - val_loss: 0.3171
Epoch 3/200
24/24 [==============================] - 4s 148ms/step - loss: 0.2905 - val_loss: 0.2981
Epoch 4/200
24/24 [==============================] - 4s 148ms/step - loss: 0.2705 - val_loss: 0.2805
Epoch 5/200
24/24 [==============================] - 4s 151ms/step - loss: 0.2545 - val_loss: 0.2612
Epoch 6/200
24/24 [==============================] - 4s 153ms/step - loss: 0.2385 - val_loss: 0.2376
Epoch 7/200
24/24 [==============================] - 4s 151ms/step - loss: 0.2283 - val_loss: 0.2227
Epoch 8/200
24/24 [==============================] - 4s 151ms/step - loss: 0.2124 - val_loss: 0.2059
Epoch 9/200
24/24 [==============================] - 4s 151ms/step - loss: 0.2003 - val_loss: 0.1901
Epoch 10/200
24/24 [==============================] - 4s 149ms/step - loss: 0.1890 - val_l

Epoch 161/200
24/24 [==============================] - 3s 138ms/step - loss: 0.0338 - val_loss: 0.0099
Epoch 162/200
24/24 [==============================] - 3s 139ms/step - loss: 0.0329 - val_loss: 0.0098
Epoch 163/200
24/24 [==============================] - 3s 139ms/step - loss: 0.0341 - val_loss: 0.0094
Epoch 164/200
24/24 [==============================] - 3s 131ms/step - loss: 0.0324 - val_loss: 0.0106
Epoch 165/200
24/24 [==============================] - 3s 137ms/step - loss: 0.0329 - val_loss: 0.0104
Epoch 166/200
24/24 [==============================] - 3s 133ms/step - loss: 0.0327 - val_loss: 0.0103
Epoch 167/200
24/24 [==============================] - 3s 136ms/step - loss: 0.0332 - val_loss: 0.0127
Epoch 168/200
24/24 [==============================] - 3s 146ms/step - loss: 0.0320 - val_loss: 0.0100
Epoch 169/200
24/24 [==============================] - 3s 137ms/step - loss: 0.0321 - val_loss: 0.0116
Epoch 170/200
24/24 [==============================] - 3s 144ms/step - lo

In [21]:
#os.mkdir('RESULTS_LOSS')

In [22]:
import os
if not os.path.exists(DATA_PATH+"/../../TRANSFORMER_SCALER_DUMPS/"):
    os.mkdir(DATA_PATH+"/../../TRANSFORMER_SCALER_DUMPS/")

if not os.path.exists(DATA_PATH+"/../../TRANSFORMER_RESULTS_LOSS/"):
    os.mkdir(DATA_PATH+"/../../TRANSFORMER_RESULTS_LOSS/")

if not os.path.exists(DATA_PATH+"/../../TRANSFORMER_CHECKPOINTS/"):
    os.mkdir(DATA_PATH+"/../../TRANSFORMER_CHECKPOINTS/")


In [23]:
TEST_LOSS = transformer_predictor.evaluate(X_test, y_test)
print(TEST_LOSS)

2/2 [==============================] - 0s 74ms/step - loss: 0.0185
0.01849478855729103


In [24]:
# SAVE THE SCALER
import joblib
import os

scaler_filename = DATA_PATH+"/../../TRANSFORMER_SCALER_DUMPS/transformer_min_max_scaler_lookahead"+str(output_cardinality)+".save"
joblib.dump(scaler, scaler_filename)
# And now to load...

['../DATASET/TRANSORMER_DATA/../../TRANSFORMER_SCALER_DUMPS/transformer_min_max_scaler_lookahead20.save']

In [25]:
TRT_LOSS = stats.history['loss']
training_loss = pd.DataFrame(TRT_LOSS, columns=['TR_LOSS'])
training_loss.to_csv(DATA_PATH+"/../../TRANSFORMER_RESULTS_LOSS/TRANSFORMER_TR_LOSS_"+str(output_cardinality)+'.csv')


In [26]:
VAL_LOSS = stats.history['val_loss']
validation_loss = pd.DataFrame(VAL_LOSS, columns=['VAL_LOSS'])
validation_loss.to_csv(DATA_PATH+"/../../TRANSFORMER_RESULTS_LOSS/TRANSFORMER_VAL_LOSS_"+str(output_cardinality)+'.csv')


In [27]:
test_loss = pd.DataFrame([TEST_LOSS], columns=['TEST_LOSS'])
test_loss.to_csv(DATA_PATH+"/../../TRANSFORMER_RESULTS_LOSS/TRANSFORMER_TEST_LOSS_"+str(output_cardinality)+'.csv')

In [28]:
transformer_predictor.save(DATA_PATH+"/../../TRANSFORMER_CHECKPOINTS/transformer_lookahead"+str(output_cardinality)+".model")

INFO:tensorflow:Assets written to: ../DATASET/TRANSORMER_DATA/../../TRANSFORMER_CHECKPOINTS/transformer_lookahead20.model/assets


INFO:tensorflow:Assets written to: ../DATASET/TRANSORMER_DATA/../../TRANSFORMER_CHECKPOINTS/transformer_lookahead20.model/assets


In [29]:
#modelnew= tensorflow.keras.models.load_model(DATA_PATH+"/../../TRANSFORMER_CHECKPOINTS/transformer_lookahead"+str(output_cardinality)+".model")

In [30]:
#modelnew.predict(X_test)